# Семинар 1

## План ноутбука

1. Установка `PyTorch`
1. Введение в `PyTorch`
1. Полносвязные слои и функции активации в `PyTorch`
1. Градиентный спуск своими руками

## Установка `PyTorch`

Мы будем использовать библиотеку для глубинного обучения `PyTorch`, ее можно не устанавливать, можно пользоваться сайтами [Kaggle](kaggle.com) и [Google Colab](colab.research.google.com/) для обучения в облаке (или с учителем?).

Чтобы установить `PyTorch` локально себе на компьютер нужно ответить на два вопроса - какая у вас операционная система и есть ли у вас дискретная видеокарта (GPU) и если есть, то какого производителя. В зависимости от ваших ответов мы получаем три варианта по операционной системе - Linux, Mac и Windows; три варианта по дискретной видеокарте - нет видеокарты (доступен только центральный процессор CPU), есть видеокарта от Nvidia или есть видеокарта от AMD (это производитель именно чипа, конечный вендор может быть другой, например, ASUS, MSI, Palit). Работа с PyTorch с видеокартой от AMD это экзотика, которая выходит за рамки нашего курса, поэтому рассмотрим только варианты *нет видеокарты*/*есть видеокарта от Nvidia*.


Выберите на [сайте](https://pytorch.org/get-started/locally/) подходящие вам варианты операционной системы/видеокарты и скопируйте команду для установки. Разберем подробно самые популярные варианты установки:

### Установка в Linux ([поддерживаемые дистрибутивы](https://pytorch.org/get-started/locally/#supported-linux-distributions))

На линуксе будет работать поддержка `PyTorch` в любой конфигурации, что у вас нет видеокарты, что есть от Nvidia, что от AMD.

Пререквизит для работы с видеокартой от Nvidia - нужно поставить CUDA, это инструмент от компании Nvidia, который позволяет ускорять вычисления на их же ГПУ. Чтобы поставить себе на машину все правильно воспользуйтесь этим [гайдом](https://docs.nvidia.com/cuda/cuda-installation-guide-linux/index.html) от Nvidia.

 - **pip**

`pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cpu` для тех, у кого нет видеокарты.

`pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118` для тех, у кого есть видеокарта (либо другой `--extra-index-url`, смотрите на сайте PyTorch, в зависимости от версии CUDA).

 - **conda**

`conda install pytorch torchvision torchaudio cpuonly -c pytorch` для тех, у кого нет видеокарты.

`conda install pytorch torchvision torchaudio cudatoolkit=11.8 -c pytorch -c conda-forge` для тех, у кого есть видеокарта (либо немного другая команда, в зависимости от версии CUDA).

### Установка в Windows

На винде будет работать поддержка `PyTorch` только для видеокарт от Nvidia и без видеокарт вообще.

Пререквизит для работы с видеокартой от Nvidia - нужно поставить CUDA, это инструмент от компании Nvidia, который позволяет ускорять вычисления на их же ГПУ. Чтобы поставить себе на машину все правильно воспользуйтесь этим [гайдом](https://docs.nvidia.com/cuda/cuda-installation-guide-microsoft-windows/index.html) от Nvidia.

 - **pip**

`pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cpu` для тех, у кого нет видеокарты.

`pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118` для тех, у кого есть видеокарта (либо другой `--extra-index-url`, смотрите на сайте PyTorch, в зависимости от версии CUDA).

 - **conda**

`conda install pytorch torchvision torchaudio cpuonly -c pytorch` для тех, у кого нет видеокарты.

`conda install pytorch torchvision torchaudio cudatoolkit=11.8 -c pytorch -c conda-forge` для тех, у кого есть видеокарта (либо немного другая команда, в зависимости от версии CUDA).



### Установка на Mac

На маках есть поддержка `PyTorch` как на CPU, так и на GPU, но только для Apple Silicon, то есть на чипах M1, M2 и так далее.

При этом поддержка ускорения с помощью GPU есть только для версий макоси выше 12.3: MPS acceleration is available on MacOS 12.3+

 - **pip**

`pip3 install torch torchvision torchaudio`

 - **conda**

`conda install pytorch::pytorch torchvision torchaudio -c pytorch`

## Введение в `PyTorch`

### Тензоры

Тензоры — это специализированная структура данных, по сути это массивы и матрицы. Тензоры очень похожи на массивы в numpy, так что, если у вас хорошо с numpy, то разобраться в PyTorch тензорах будет очень просто. В PyTorch мы используем тензоры для кодирования входных и выходных данных модели, а также параметров модели.

In [1]:
import torch
import numpy as np

### Создание тензоров

Тензор можно создать напрямую из каких-то данных - нам подходят все списки с числами:

In [2]:
some_data = [1, 2, 3, 4]
some_tensor = torch.tensor(some_data)

some_tensor

tensor([1, 2, 3, 4])

In [3]:
some_data = [[1, 2], [3, 4], [5, 6]]
some_tensor = torch.tensor(some_data)

some_tensor

tensor([[1, 2],
        [3, 4],
        [5, 6]])

In [4]:
some_data = [[[1], [2]], [[3], [4]], [[5], [6]]]
some_tensor = torch.tensor(some_data)

some_tensor

tensor([[[1],
         [2]],

        [[3],
         [4]],

        [[5],
         [6]]])

На самом деле про "все" списки с числами - обман. Если у вашего списка есть какой-то уровень вложенности, то должны совпадать размерности у всех вложенных списков (подробнее про размерности поговорим позже):

In [5]:
some_other_data = [[1, 2, 2], [3, 4, 3], [5, 6, 7]]
some_other_tensor = torch.tensor(some_other_data)

some_other_tensor

tensor([[1, 2, 2],
        [3, 4, 3],
        [5, 6, 7]])

In [7]:
some_other_data = [[1, 2], [3, 4], [5, 6]]
some_other_tensor = torch.tensor(some_other_data)

some_other_tensor

tensor([[1, 2],
        [3, 4],
        [5, 6]])

Также тензоры можно создавать из numpy массивов и наоборот:

In [8]:
some_numpy_array = np.array(some_data)

some_numpy_array

array([[[1],
        [2]],

       [[3],
        [4]],

       [[5],
        [6]]])

In [9]:
some_tensor_from_numpy = torch.from_numpy(some_numpy_array)

some_tensor_from_numpy

tensor([[[1],
         [2]],

        [[3],
         [4]],

        [[5],
         [6]]])

При этом если мы создаем тензор из numpy массива с помощью `torch.from_numpy`, то они делят между собой память, где лежат их данные и, соответственно, при изменении тензора меняется numpy массив и наоборот:

In [10]:
x = np.ones(10)
y = torch.from_numpy(x)

x, y

(array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=torch.float64))

In [11]:
x += 1

x, y

(array([2., 2., 2., 2., 2., 2., 2., 2., 2., 2.]),
 tensor([2., 2., 2., 2., 2., 2., 2., 2., 2., 2.], dtype=torch.float64))

In [12]:
x = torch.ones(10)
y = x.numpy()

x, y

(tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32))

In [13]:
x += 1

x, y

(tensor([2., 2., 2., 2., 2., 2., 2., 2., 2., 2.]),
 array([2., 2., 2., 2., 2., 2., 2., 2., 2., 2.], dtype=float32))

In [16]:
x = np.ones(10)
y = torch.tensor(x)

x, y

(array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=torch.float64))

In [17]:
y += 1

x, y

(array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 tensor([2., 2., 2., 2., 2., 2., 2., 2., 2., 2.], dtype=torch.float64))

Можем создать тензор со случайными или константными значениями:

In [18]:
?torch.rand

In [ ]:
shape = (2, 3)

random_tensor = torch.rand(shape)
ones_tensor = torch.ones(shape)
zeros_tensor = torch.zeros(shape)
empty_tensor = torch.empty(shape)

random_tensor, ones_tensor, zeros_tensor, empty_tensor

(tensor([[0.8877, 0.7999, 0.1586],
         [0.0426, 0.9348, 0.4901]]),
 tensor([[1., 1., 1.],
         [1., 1., 1.]]),
 tensor([[0., 0., 0.],
         [0., 0., 0.]]),
 tensor([[0., 0., 0.],
         [0., 0., 0.]]))

Теперь поговорим про размерности подробнее.

У тензора есть какой-то размер, какая форма. Первое с чем нужно определиться, какой **размерности** тензор - количество осей у него.

In [19]:
shape = (10)  # одна ось (вектор)

tensor = torch.rand(shape)

tensor

tensor([0.6537, 0.9629, 0.0803, 0.8285, 0.1926, 0.0446, 0.6284, 0.7663, 0.4429,
        0.6711])

In [20]:
shape = (2, 3)  # две оси (матрица)

tensor = torch.rand(shape)

tensor

tensor([[0.1924, 0.9616, 0.6403],
        [0.7661, 0.6347, 0.7017]])

In [21]:
shape = (3, 2, 3)  # три оси (и больше - тензор)

tensor = torch.rand(shape)

tensor

tensor([[[0.8422, 0.6300, 0.5070],
         [0.8698, 0.2860, 0.3171]],

        [[0.1826, 0.4007, 0.4874],
         [0.0406, 0.4140, 0.6837]],

        [[0.2089, 0.3730, 0.3101],
         [0.1159, 0.0253, 0.9788]]])

Тензор с размерностью 1 - это просто вектор, список чисел.

Тензор с размерностью 2 - это просто матрица, то есть список списков чисел.

Тензор с размерностью 3 и больше - это тензор, то есть список списков списков ... чисел.

![](meme.png)

Получить доступ к размеру уже созданного тензора - метод `.shape`:

In [22]:
some_data = [[[1], [2]], [[3], [4]], [[5], [6]]]
some_tensor = torch.tensor(some_data)

print(some_tensor)
print(some_tensor.shape)

tensor([[[1],
         [2]],

        [[3],
         [4]],

        [[5],
         [6]]])
torch.Size([3, 2, 1])


Давайте сделаем тензор, который будет нам имитировать изображение - сделаем его размер `(c, h, w)`, где `h` и `w` это его высота и ширина, а `c` - число каналов в цветовом пространстве (в черно-белом 1, в RGB 3):

In [23]:
h = 9
w = 16
c = 3

shape = (c, h, w)

image_tensor = torch.rand(shape)

image_tensor

tensor([[[7.9468e-01, 6.1469e-01, 8.2209e-01, 6.4438e-01, 6.2663e-01,
          6.5292e-02, 6.5813e-01, 8.1140e-01, 5.8233e-01, 5.5889e-01,
          3.1757e-01, 3.8509e-01, 9.6519e-01, 3.1608e-01, 2.2355e-01,
          9.4104e-01],
         [2.0473e-01, 9.3400e-04, 7.5946e-01, 7.6341e-01, 5.4445e-01,
          1.0155e-01, 3.8065e-01, 7.3055e-02, 3.6887e-01, 7.1746e-01,
          8.6635e-01, 8.3288e-01, 7.9785e-01, 4.1817e-01, 6.8690e-01,
          2.1260e-02],
         [9.7963e-02, 4.7697e-01, 8.6817e-01, 3.8058e-01, 1.8445e-01,
          2.2915e-01, 6.2268e-01, 8.5849e-01, 3.4141e-01, 6.1160e-01,
          7.7932e-01, 5.1471e-01, 2.9338e-01, 3.9142e-01, 8.3034e-01,
          3.1170e-01],
         [9.1439e-01, 6.8076e-01, 4.8710e-01, 5.2645e-01, 7.7575e-01,
          3.4801e-01, 7.8233e-01, 5.3238e-01, 9.3325e-01, 1.3442e-01,
          6.9308e-01, 9.9791e-01, 8.0225e-01, 2.1435e-01, 3.7137e-01,
          4.8055e-01],
         [9.4338e-02, 4.2397e-01, 4.3697e-01, 8.1516e-01, 1.2389e-02

In [24]:
image_tensor.shape

torch.Size([3, 9, 16])

Можем попробовать поменять размер тензора, например, [вытянуть его в вектор](https://pytorch.org/docs/stable/generated/torch.ravel.html):

In [25]:
image_tensor.flatten()

tensor([7.9468e-01, 6.1469e-01, 8.2209e-01, 6.4438e-01, 6.2663e-01, 6.5292e-02,
        6.5813e-01, 8.1140e-01, 5.8233e-01, 5.5889e-01, 3.1757e-01, 3.8509e-01,
        9.6519e-01, 3.1608e-01, 2.2355e-01, 9.4104e-01, 2.0473e-01, 9.3400e-04,
        7.5946e-01, 7.6341e-01, 5.4445e-01, 1.0155e-01, 3.8065e-01, 7.3055e-02,
        3.6887e-01, 7.1746e-01, 8.6635e-01, 8.3288e-01, 7.9785e-01, 4.1817e-01,
        6.8690e-01, 2.1260e-02, 9.7963e-02, 4.7697e-01, 8.6817e-01, 3.8058e-01,
        1.8445e-01, 2.2915e-01, 6.2268e-01, 8.5849e-01, 3.4141e-01, 6.1160e-01,
        7.7932e-01, 5.1471e-01, 2.9338e-01, 3.9142e-01, 8.3034e-01, 3.1170e-01,
        9.1439e-01, 6.8076e-01, 4.8710e-01, 5.2645e-01, 7.7575e-01, 3.4801e-01,
        7.8233e-01, 5.3238e-01, 9.3325e-01, 1.3442e-01, 6.9308e-01, 9.9791e-01,
        8.0225e-01, 2.1435e-01, 3.7137e-01, 4.8055e-01, 9.4338e-02, 4.2397e-01,
        4.3697e-01, 8.1516e-01, 1.2389e-02, 8.2236e-01, 4.7251e-01, 6.0982e-01,
        7.6519e-01, 2.4980e-01, 3.0524e-

In [26]:
image_tensor.ravel()

tensor([7.9468e-01, 6.1469e-01, 8.2209e-01, 6.4438e-01, 6.2663e-01, 6.5292e-02,
        6.5813e-01, 8.1140e-01, 5.8233e-01, 5.5889e-01, 3.1757e-01, 3.8509e-01,
        9.6519e-01, 3.1608e-01, 2.2355e-01, 9.4104e-01, 2.0473e-01, 9.3400e-04,
        7.5946e-01, 7.6341e-01, 5.4445e-01, 1.0155e-01, 3.8065e-01, 7.3055e-02,
        3.6887e-01, 7.1746e-01, 8.6635e-01, 8.3288e-01, 7.9785e-01, 4.1817e-01,
        6.8690e-01, 2.1260e-02, 9.7963e-02, 4.7697e-01, 8.6817e-01, 3.8058e-01,
        1.8445e-01, 2.2915e-01, 6.2268e-01, 8.5849e-01, 3.4141e-01, 6.1160e-01,
        7.7932e-01, 5.1471e-01, 2.9338e-01, 3.9142e-01, 8.3034e-01, 3.1170e-01,
        9.1439e-01, 6.8076e-01, 4.8710e-01, 5.2645e-01, 7.7575e-01, 3.4801e-01,
        7.8233e-01, 5.3238e-01, 9.3325e-01, 1.3442e-01, 6.9308e-01, 9.9791e-01,
        8.0225e-01, 2.1435e-01, 3.7137e-01, 4.8055e-01, 9.4338e-02, 4.2397e-01,
        4.3697e-01, 8.1516e-01, 1.2389e-02, 8.2236e-01, 4.7251e-01, 6.0982e-01,
        7.6519e-01, 2.4980e-01, 3.0524e-

In [27]:
image_tensor.ravel().shape

torch.Size([432])

In [28]:
h * w * c

432

Посчитаем количество элементов в тензоре с помощью [специальной функции](https://pytorch.org/docs/stable/generated/torch.numel.html):

In [29]:
image_tensor.numel()

432

In [30]:
h = 2
w = 3
c = 3

shape = (c, h, w)

image_tensor = torch.rand(shape)

image_tensor

tensor([[[0.2181, 0.8573, 0.8212],
         [0.7708, 0.4495, 0.7213]],

        [[0.0087, 0.3256, 0.3137],
         [0.1772, 0.3231, 0.5066]],

        [[0.6679, 0.4625, 0.5122],
         [0.1940, 0.8536, 0.2280]]])

Попробуем поменять размер с помощью функции [reshape](https://pytorch.org/docs/stable/generated/torch.reshape.html#torch.reshape):

In [31]:
image_tensor.reshape(c, h * w)

tensor([[0.2181, 0.8573, 0.8212, 0.7708, 0.4495, 0.7213],
        [0.0087, 0.3256, 0.3137, 0.1772, 0.3231, 0.5066],
        [0.6679, 0.4625, 0.5122, 0.1940, 0.8536, 0.2280]])

Попробуем собрать из нескольких тензоров один большой:

[torch.cat](https://pytorch.org/docs/stable/generated/torch.cat.html#torch.cat)

In [32]:
x = torch.randn(2, 3)

In [33]:
x

tensor([[ 0.4755, -1.5607,  0.2557],
        [ 1.0482,  0.8567,  0.4277]])

In [34]:
torch.cat((x, x, x), dim=0)

tensor([[ 0.4755, -1.5607,  0.2557],
        [ 1.0482,  0.8567,  0.4277],
        [ 0.4755, -1.5607,  0.2557],
        [ 1.0482,  0.8567,  0.4277],
        [ 0.4755, -1.5607,  0.2557],
        [ 1.0482,  0.8567,  0.4277]])

In [35]:
torch.stack((x, x, x, x), dim=0).shape

torch.Size([4, 2, 3])

In [36]:
x = torch.randn(3, 3, 2)
y = torch.randn(5, 3, 2)
z = torch.randn(1, 3, 2)

for tensor in [x, y, z]:
    print(tensor)

torch.cat((x, y, z), dim=0)

tensor([[[-0.7820,  1.1845],
         [-0.1435,  1.1591],
         [-0.2662, -0.0421]],

        [[-0.3579,  0.1191],
         [ 0.7933,  0.5395],
         [-0.6335, -0.0982]],

        [[-1.3771, -1.2553],
         [ 0.0693, -0.3803],
         [-0.2658,  0.6644]]])
tensor([[[ 1.6604, -0.6969],
         [-0.4091, -0.4232],
         [-1.4679, -0.3336]],

        [[ 0.7284,  2.1856],
         [-0.2785, -1.1647],
         [ 1.0323, -0.5826]],

        [[-0.7928,  0.5952],
         [ 0.6722, -2.0667],
         [-1.0109,  0.8838]],

        [[-0.6779, -2.8769],
         [-0.5999,  0.6378],
         [-0.0760, -0.5308]],

        [[-0.1381,  0.2921],
         [ 0.6438, -0.6715],
         [ 0.3714, -0.1394]]])
tensor([[[-1.8790,  0.4378],
         [-0.8440,  1.7307],
         [-0.3882, -0.1971]]])


tensor([[[-0.7820,  1.1845],
         [-0.1435,  1.1591],
         [-0.2662, -0.0421]],

        [[-0.3579,  0.1191],
         [ 0.7933,  0.5395],
         [-0.6335, -0.0982]],

        [[-1.3771, -1.2553],
         [ 0.0693, -0.3803],
         [-0.2658,  0.6644]],

        [[ 1.6604, -0.6969],
         [-0.4091, -0.4232],
         [-1.4679, -0.3336]],

        [[ 0.7284,  2.1856],
         [-0.2785, -1.1647],
         [ 1.0323, -0.5826]],

        [[-0.7928,  0.5952],
         [ 0.6722, -2.0667],
         [-1.0109,  0.8838]],

        [[-0.6779, -2.8769],
         [-0.5999,  0.6378],
         [-0.0760, -0.5308]],

        [[-0.1381,  0.2921],
         [ 0.6438, -0.6715],
         [ 0.3714, -0.1394]],

        [[-1.8790,  0.4378],
         [-0.8440,  1.7307],
         [-0.3882, -0.1971]]])

In [37]:
x = torch.randn(2, 3)
y = torch.randn(2, 5)
z = torch.randn(2, 1)

for tensor in [x, y, z]:
    print(tensor)

torch.stack((x, x, x), dim=0)

tensor([[-2.0031, -2.9331, -0.1523],
        [-1.2373, -2.6953, -0.3506]])
tensor([[ 0.3864,  0.9333, -0.7854, -0.4726, -1.1477],
        [ 0.3865,  0.1644, -0.6890,  0.2751,  0.3463]])
tensor([[-0.5671],
        [-0.4113]])


tensor([[[-2.0031, -2.9331, -0.1523],
         [-1.2373, -2.6953, -0.3506]],

        [[-2.0031, -2.9331, -0.1523],
         [-1.2373, -2.6953, -0.3506]],

        [[-2.0031, -2.9331, -0.1523],
         [-1.2373, -2.6953, -0.3506]]])

Теперь добавим дополнительную ось:

[torch.unsqueeze](https://pytorch.org/docs/stable/generated/torch.unsqueeze.html)

In [38]:
x = torch.rand(2, 3)

print(x)
print()
print(x.unsqueeze(0), x.unsqueeze(0).shape)
print()
print(x.unsqueeze(1), x.unsqueeze(1).shape)
print()
print(x.unsqueeze(2), x.unsqueeze(2).shape)

tensor([[0.3811, 0.0757, 0.7001],
        [0.9836, 0.3984, 0.7788]])

tensor([[[0.3811, 0.0757, 0.7001],
         [0.9836, 0.3984, 0.7788]]]) torch.Size([1, 2, 3])

tensor([[[0.3811, 0.0757, 0.7001]],

        [[0.9836, 0.3984, 0.7788]]]) torch.Size([2, 1, 3])

tensor([[[0.3811],
         [0.0757],
         [0.7001]],

        [[0.9836],
         [0.3984],
         [0.7788]]]) torch.Size([2, 3, 1])


Уберем лишние оси (где размер единичка):

In [41]:
x = torch.rand(1, 2, 1, 3)

print(x)
print()
print(x.squeeze(), x.squeeze().shape)
print()
print(x.squeeze(2), x.squeeze(2).shape)
print()
print(x.squeeze(1), x.squeeze(1).shape)

tensor([[[[0.4897, 0.2539, 0.3307]],

         [[0.1887, 0.3866, 0.4316]]]])

tensor([[0.4897, 0.2539, 0.3307],
        [0.1887, 0.3866, 0.4316]]) torch.Size([2, 3])

tensor([[[0.4897, 0.2539, 0.3307],
         [0.1887, 0.3866, 0.4316]]]) torch.Size([1, 2, 3])

tensor([[[[0.4897, 0.2539, 0.3307]],

         [[0.1887, 0.3866, 0.4316]]]]) torch.Size([1, 2, 1, 3])


Теперь поговорим про типы данных в тензорах. По умолчанию в тензорах лежат числа в torch.float32 для вещественных и torch.int64 для целочисленных.

In [42]:
tensor = torch.tensor([1.5, 2.2, 3.7, 4.9])

tensor

tensor([1.5000, 2.2000, 3.7000, 4.9000])

In [43]:
tensor.dtype

torch.float32

In [44]:
tensor = torch.tensor([1.5, 2.2, 3.7, 4.9], dtype=torch.float16)

tensor

tensor([1.5000, 2.1992, 3.6992, 4.8984], dtype=torch.float16)

In [45]:
tensor = torch.tensor([1.5, 2.2, 3.7, 4.9], dtype=torch.float64)

tensor

tensor([1.5000, 2.2000, 3.7000, 4.9000], dtype=torch.float64)

In [46]:
tensor = torch.tensor([15, 22, 37, 49])

tensor

tensor([15, 22, 37, 49])

In [47]:
tensor.dtype

torch.int64

In [48]:
tensor = torch.tensor([15, 22, 37, 49], dtype=torch.int32)

tensor

tensor([15, 22, 37, 49], dtype=torch.int32)

In [49]:
tensor = torch.tensor([15, 22, 37, 49], dtype=torch.int16)

tensor

tensor([15, 22, 37, 49], dtype=torch.int16)

Размещение тензора на GPU:

In [50]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name())

True
Tesla T4


In [51]:
! nvidia-smi

Mon Mar  3 17:13:55 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   47C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [54]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device)

cuda:0


In [55]:
tensor = torch.tensor([15, 22, 37, 49], device=device)

tensor

tensor([15, 22, 37, 49], device='cuda:0')

In [56]:
tensor = torch.tensor([15, 22, 37, 49])

print(tensor)

tensor = tensor.to(device)

tensor

tensor([15, 22, 37, 49])


tensor([15, 22, 37, 49], device='cuda:0')

In [57]:
tensor.to(torch.int32)

tensor([15, 22, 37, 49], device='cuda:0', dtype=torch.int32)

In [58]:
tensor = tensor.cpu()

tensor

tensor([15, 22, 37, 49])

In [59]:
tensor.cuda()

tensor([15, 22, 37, 49], device='cuda:0')

In [61]:
a = torch.rand(2, 3)
b = torch.rand(2, 3)

a, b, a + b

(tensor([[0.8241, 0.0582, 0.4713],
         [0.7449, 0.7170, 0.8571]]),
 tensor([[0.7316, 0.4450, 0.2000],
         [0.8295, 0.8859, 0.8563]]),
 tensor([[1.5557, 0.5033, 0.6713],
         [1.5743, 1.6029, 1.7134]]))

In [62]:
a = a.to(device)

a + b

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [63]:
b = b.to(device)

a + b

tensor([[1.5557, 0.5033, 0.6713],
        [1.5743, 1.6029, 1.7134]], device='cuda:0')

### Операции с тензорами

Большая часть операций с тензорами хорошо описана в их [документации](https://pytorch.org/docs/stable/torch.html), разберем основные:

In [64]:
a = torch.rand(2, 3)
b = torch.rand(2, 3)

a, b

(tensor([[0.0409, 0.1142, 0.6275],
         [0.8805, 0.5349, 0.0824]]),
 tensor([[0.1867, 0.2666, 0.6386],
         [0.1872, 0.7583, 0.9260]]))

In [66]:
# поэлементные

print(a + b)

print()

print(torch.add(a, b))

print()

print(a.add(b))

tensor([[0.2276, 0.3807, 1.2661],
        [1.0676, 1.2932, 1.0085]])

tensor([[0.2276, 0.3807, 1.2661],
        [1.0676, 1.2932, 1.0085]])

tensor([[0.2276, 0.3807, 1.2661],
        [1.0676, 1.2932, 1.0085]])


In [67]:
print(a - b)

print()

print(torch.sub(a, b))

print()

print(a.sub(b))

tensor([[-0.1458, -0.1524, -0.0111],
        [ 0.6933, -0.2235, -0.8436]])

tensor([[-0.1458, -0.1524, -0.0111],
        [ 0.6933, -0.2235, -0.8436]])

tensor([[-0.1458, -0.1524, -0.0111],
        [ 0.6933, -0.2235, -0.8436]])


In [68]:
print(a * b)

print()

print(torch.mul(a, b))

print()

print(a.mul(b))

tensor([[0.0076, 0.0304, 0.4007],
        [0.1648, 0.4056, 0.0764]])

tensor([[0.0076, 0.0304, 0.4007],
        [0.1648, 0.4056, 0.0764]])

tensor([[0.0076, 0.0304, 0.4007],
        [0.1648, 0.4056, 0.0764]])


In [69]:
print(a / b)

print()

print(torch.div(a, b))

print()

print(a.div(b))

tensor([[0.2190, 0.4284, 0.9827],
        [4.7044, 0.7053, 0.0890]])

tensor([[0.2190, 0.4284, 0.9827],
        [4.7044, 0.7053, 0.0890]])

tensor([[0.2190, 0.4284, 0.9827],
        [4.7044, 0.7053, 0.0890]])


In [70]:
a = torch.rand(2, 3)
b = torch.rand(3, 4)
c = torch.rand(5, 5)

a, b, c

(tensor([[0.4312, 0.6839, 0.3543],
         [0.6584, 0.0167, 0.1576]]),
 tensor([[0.4156, 0.0237, 0.4103, 0.4978],
         [0.9251, 0.2084, 0.5384, 0.8898],
         [0.9128, 0.7157, 0.9270, 0.5119]]),
 tensor([[0.5552, 0.4834, 0.8817, 0.3200, 0.1444],
         [0.5312, 0.4177, 0.4253, 0.9626, 0.0145],
         [0.1837, 0.6778, 0.9246, 0.1530, 0.6398],
         [0.6803, 0.0245, 0.8865, 0.6881, 0.1617],
         [0.4758, 0.0965, 0.9625, 0.1711, 0.0645]]))

In [71]:
# матричные операции

print(a @ b, (a @ b).shape)

print()

print(torch.matmul(a, b), torch.matmul(a, b).shape)

print()

print(c.trace())

print()

print(c.exp())

tensor([[1.1353, 0.4063, 0.8736, 1.0045],
        [0.4330, 0.1319, 0.4253, 0.4232]]) torch.Size([2, 4])

tensor([[1.1353, 0.4063, 0.8736, 1.0045],
        [0.4330, 0.1319, 0.4253, 0.4232]]) torch.Size([2, 4])

tensor(2.6502)

tensor([[1.7424, 1.6216, 2.4149, 1.3771, 1.1553],
        [1.7010, 1.5184, 1.5301, 2.6184, 1.0146],
        [1.2016, 1.9695, 2.5210, 1.1653, 1.8960],
        [1.9744, 1.0248, 2.4266, 1.9899, 1.1755],
        [1.6094, 1.1013, 2.6181, 1.1867, 1.0666]])


### [Автоматическое дифференцирование](https://pytorch.org/docs/stable/notes/autograd.html)

In [72]:
x = torch.rand(5)

x

tensor([0.2568, 0.9172, 0.8618, 0.5437, 0.0743])

In [73]:
w = torch.rand(3, 5, requires_grad=True)

w

tensor([[0.7815, 0.6583, 0.7194, 0.7992, 0.9202],
        [0.3518, 0.8387, 0.4258, 0.4919, 0.7183],
        [0.8379, 0.7030, 0.6740, 0.8276, 0.4316]], requires_grad=True)

In [74]:
print(w.grad)

None


In [75]:
first_z = torch.empty(3)

first_z

tensor([ 1.7660e-28,  0.0000e+00, -2.0682e-29])

In [76]:
for i in range(3):
    first_z[i] = torch.sum(w[i] * x)

first_z

tensor([1.9273, 1.5474, 1.9228], grad_fn=<CopySlices>)

In [77]:
w

tensor([[0.7815, 0.6583, 0.7194, 0.7992, 0.9202],
        [0.3518, 0.8387, 0.4258, 0.4919, 0.7183],
        [0.8379, 0.7030, 0.6740, 0.8276, 0.4316]], requires_grad=True)

In [79]:
w.t(), w.T

(tensor([[0.7815, 0.3518, 0.8379],
         [0.6583, 0.8387, 0.7030],
         [0.7194, 0.4258, 0.6740],
         [0.7992, 0.4919, 0.8276],
         [0.9202, 0.7183, 0.4316]], grad_fn=<TBackward0>),
 tensor([[0.7815, 0.3518, 0.8379],
         [0.6583, 0.8387, 0.7030],
         [0.7194, 0.4258, 0.6740],
         [0.7992, 0.4919, 0.8276],
         [0.9202, 0.7183, 0.4316]], grad_fn=<PermuteBackward0>))

In [80]:
z = torch.matmul(x, w.t())

z

tensor([1.9273, 1.5474, 1.9228], grad_fn=<SqueezeBackward4>)

In [81]:
v = torch.rand(3, requires_grad=True)

v

tensor([0.7200, 0.1701, 0.8342], requires_grad=True)

In [82]:
print(v.grad)

None


In [83]:
y = torch.sum(z * v)

y

tensor(3.2548, grad_fn=<SumBackward0>)

In [84]:
y.item()

3.254817247390747

In [85]:
loss = torch.mean((y - 2) ** 2)

In [86]:
loss

tensor(1.5746, grad_fn=<MeanBackward0>)

In [87]:
print(f'{x.grad=}\n')
print(f'{w.grad=}\n')
print(f'{z.grad=}\n')
print(f'{v.grad=}\n')

x.grad=None

w.grad=None

z.grad=None

v.grad=None



<ipython-input-87-b1828bd1c118>:3: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at aten/src/ATen/core/TensorBody.h:489.)
  print(f'{z.grad=}\n')


In [88]:
loss.backward()

In [89]:
print(f'{x.grad=}\n')
print(f'{w.grad=}\n')
print(f'{z.grad=}\n')
print(f'{v.grad=}\n')

x.grad=None

w.grad=tensor([[0.4640, 1.6572, 1.5570, 0.9823, 0.1343],
        [0.1096, 0.3916, 0.3680, 0.2321, 0.0317],
        [0.5376, 1.9202, 1.8041, 1.1382, 0.1556]])

z.grad=None

v.grad=tensor([4.8368, 3.8834, 4.8255])



<ipython-input-89-b1828bd1c118>:3: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at aten/src/ATen/core/TensorBody.h:489.)
  print(f'{z.grad=}\n')


In [90]:
a = torch.rand(1, requires_grad=True)
b = torch.rand(1, requires_grad=True)

a, b

(tensor([0.3538], requires_grad=True), tensor([0.1641], requires_grad=True))

In [91]:
loss = (a - b)

loss

tensor([0.1897], grad_fn=<SubBackward0>)

In [92]:
print(f'{a.grad=}\n')
print(f'{b.grad=}\n')

a.grad=None

b.grad=None



In [93]:
loss.backward()

In [94]:
print(f'{a.grad=}\n')  # 1
print(f'{b.grad=}\n')  # -1

a.grad=tensor([1.])

b.grad=tensor([-1.])



In [95]:
a.grad.zero_()
b.grad.zero_()

tensor([0.])

In [96]:
loss = (a - b) ** 2

loss

tensor([0.0360], grad_fn=<PowBackward0>)

In [97]:
print(f'{a.grad=}\n')
print(f'{b.grad=}\n')

a.grad=tensor([0.])

b.grad=tensor([0.])



In [98]:
loss.backward()

In [99]:
print(f'{a.grad=}\n')  # 2 * (a - b)
print(f'{b.grad=}\n')  # -2 * (a - b)

a.grad=tensor([0.3794])

b.grad=tensor([-0.3794])



In [100]:
2 * (a - b)

tensor([0.3794], grad_fn=<MulBackward0>)

In [101]:
a = torch.rand(3, 5, requires_grad=True)
b = torch.rand(3, 5, requires_grad=True)

a, b

(tensor([[0.5001, 0.0101, 0.8772, 0.3703, 0.1962],
         [0.9574, 0.3908, 0.9842, 0.0670, 0.8512],
         [0.2966, 0.6977, 0.1117, 0.1228, 0.6267]], requires_grad=True),
 tensor([[0.8240, 0.9718, 0.1208, 0.4451, 0.3602],
         [0.2963, 0.7087, 0.9369, 0.6724, 0.4623],
         [0.3977, 0.7756, 0.7564, 0.9802, 0.7964]], requires_grad=True))

In [102]:
loss = torch.mean(a * b)

loss

tensor(0.2699, grad_fn=<MeanBackward0>)

In [103]:
print(f'{a.grad=}\n')
print(f'{b.grad=}\n')

a.grad=None

b.grad=None



In [104]:
loss.backward()

In [105]:
print(f'{a.grad=}\n')  # b / (3 * 5)
print(f'{b.grad=}\n')  # a / (3 * 5)

a.grad=tensor([[0.0549, 0.0648, 0.0081, 0.0297, 0.0240],
        [0.0198, 0.0472, 0.0625, 0.0448, 0.0308],
        [0.0265, 0.0517, 0.0504, 0.0653, 0.0531]])

b.grad=tensor([[0.0333, 0.0007, 0.0585, 0.0247, 0.0131],
        [0.0638, 0.0261, 0.0656, 0.0045, 0.0567],
        [0.0198, 0.0465, 0.0074, 0.0082, 0.0418]])



In [106]:
a / 15

tensor([[0.0333, 0.0007, 0.0585, 0.0247, 0.0131],
        [0.0638, 0.0261, 0.0656, 0.0045, 0.0567],
        [0.0198, 0.0465, 0.0074, 0.0082, 0.0418]], grad_fn=<DivBackward0>)

In [107]:
b / 15

tensor([[0.0549, 0.0648, 0.0081, 0.0297, 0.0240],
        [0.0198, 0.0472, 0.0625, 0.0448, 0.0308],
        [0.0265, 0.0517, 0.0504, 0.0653, 0.0531]], grad_fn=<DivBackward0>)

In [108]:
a = torch.rand(3, 5, requires_grad=True)

print(f'{a=}\n')

loss1 = torch.sum(a ** 2) # 2a
loss2 = torch.sum(a) # 1

print(f'{a.grad=}\n')

loss1.backward()

print(f'{a.grad=}\n')

loss2.backward()

print(f'{a.grad=}\n')

a=tensor([[0.7625, 0.4433, 0.6503, 0.4294, 0.9976],
        [0.4784, 0.7692, 0.8731, 0.4453, 0.3291],
        [0.5476, 0.2298, 0.4109, 0.7596, 0.7950]], requires_grad=True)

a.grad=None

a.grad=tensor([[1.5250, 0.8865, 1.3006, 0.8588, 1.9953],
        [0.9569, 1.5384, 1.7462, 0.8905, 0.6582],
        [1.0953, 0.4596, 0.8217, 1.5192, 1.5900]])

a.grad=tensor([[2.5250, 1.8865, 2.3006, 1.8588, 2.9953],
        [1.9569, 2.5384, 2.7462, 1.8905, 1.6582],
        [2.0953, 1.4596, 1.8217, 2.5192, 2.5900]])



In [109]:
a = torch.rand(3, 5, requires_grad=True)
b = torch.rand(3, 5, requires_grad=False)

a, b

(tensor([[0.4486, 0.0439, 0.2433, 0.9543, 0.8724],
         [0.8447, 0.0614, 0.7515, 0.2198, 0.1312],
         [0.7981, 0.8069, 0.0999, 0.1801, 0.0466]], requires_grad=True),
 tensor([[0.7957, 0.2732, 0.1054, 0.9195, 0.7531],
         [0.1380, 0.9752, 0.5347, 0.6944, 0.5252],
         [0.0083, 0.2526, 0.0868, 0.6842, 0.6504]]))

In [110]:
loss = torch.sum(a - b)

loss

tensor(-0.8939, grad_fn=<SumBackward0>)

In [111]:
print(f'{a.grad=}\n')
print(f'{b.grad=}\n')

a.grad=None

b.grad=None



In [112]:
loss.backward()

In [113]:
print(f'{a.grad=}\n')  # all ones
print(f'{b.grad=}\n')  # None

a.grad=tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]])

b.grad=None



In [114]:
a = torch.rand(3, 5, requires_grad=True)
b = torch.rand(3, 5, requires_grad=True)

a, b

(tensor([[0.3990, 0.4893, 0.9159, 0.4451, 0.2141],
         [0.5565, 0.4390, 0.5037, 0.7966, 0.5769],
         [0.6779, 0.7874, 0.6315, 0.6979, 0.2704]], requires_grad=True),
 tensor([[0.8452, 0.2998, 0.8021, 0.1689, 0.0206],
         [0.5999, 0.8128, 0.8975, 0.4097, 0.5064],
         [0.3736, 0.3544, 0.9507, 0.8296, 0.0730]], requires_grad=True))

In [115]:
with torch.no_grad():
    loss = torch.sum(a - b)

loss

tensor(0.4568)

In [116]:
loss.backward()

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [117]:
a = torch.rand(3, 5, requires_grad=True)
b = torch.rand(3, 5, requires_grad=True)

a, b

(tensor([[0.5546, 0.9588, 0.5662, 0.1376, 0.3726],
         [0.5821, 0.9318, 0.3491, 0.7402, 0.2580],
         [0.3592, 0.3733, 0.6331, 0.0144, 0.1041]], requires_grad=True),
 tensor([[0.8023, 0.4138, 0.8898, 0.6963, 0.4560],
         [0.4862, 0.7054, 0.6507, 0.9261, 0.0337],
         [0.9956, 0.5968, 0.4448, 0.9118, 0.4559]], requires_grad=True))

In [118]:
with torch.inference_mode():
    loss = torch.sum(a - b)

loss

tensor(-2.5300)

In [119]:
loss.backward()

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [120]:
with torch.no_grad():
    a = torch.rand(3, 5, requires_grad=True)
    b = torch.rand(3, 5, requires_grad=True)

    loss = torch.sum(a + b)

    print(f'{loss=}')

    loss.backward()

loss=tensor(17.4366)


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [121]:
loss2 = torch.sum(a + b)

loss2

tensor(17.4366, grad_fn=<SumBackward0>)

In [122]:
print(f'{a.grad=}\n')
print(f'{b.grad=}\n')

a.grad=None

b.grad=None



In [123]:
loss2.backward()

In [124]:
print(f'{a.grad=}\n')
print(f'{b.grad=}\n')

a.grad=tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]])

b.grad=tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]])



In [125]:
with torch.inference_mode():
    a = torch.rand(3, 5, requires_grad=True)
    b = torch.rand(3, 5, requires_grad=True)

    loss = torch.sum(a + b)

    print(f'{loss=}')

    loss.backward()

loss=tensor(14.8938)


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [127]:
loss2 = torch.sum(a + b)

loss2

tensor(14.8938)

In [128]:
loss2.backward()

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [129]:
@torch.no_grad()
def foo():
    a = torch.rand(3, 5, requires_grad=True)
    b = torch.rand(3, 5, requires_grad=True)

    loss = torch.mean(a + b)

    print(f'{loss=}')

    return a, b

In [130]:
a, b = foo()

loss=tensor(0.9460)


In [131]:
torch.mean(a - b)

tensor(-0.0601, grad_fn=<MeanBackward0>)

In [132]:
@torch.inference_mode()
def foo():
    a = torch.rand(3, 5, requires_grad=True)
    b = torch.rand(3, 5, requires_grad=True)

    loss = torch.mean(a + b)

    print(f'{loss=}')

    return a, b

In [133]:
a, b = foo()

loss=tensor(1.0005)


In [134]:
torch.mean(a - b)

tensor(-0.0960)

In [135]:
a.requires_grad = False

## Полносвязные слои и функции активации в `PyTorch`

In [136]:
from torch import nn

### Полносвязный слой

>$y_j = \sum\limits_{i=1}^{n}x_iw_{ji} + b_j$


In [137]:
layer = nn.Linear(in_features=5, out_features=3)

In [138]:
layer

Linear(in_features=5, out_features=3, bias=True)

In [139]:
layer.weight

Parameter containing:
tensor([[ 0.1035,  0.3169, -0.1388,  0.0932, -0.3124],
        [-0.3980, -0.0908, -0.0565, -0.1986,  0.1748],
        [-0.3072, -0.2939, -0.4003,  0.1232, -0.4462]], requires_grad=True)

In [140]:
layer.weight.shape

torch.Size([3, 5])

In [141]:
layer.bias

Parameter containing:
tensor([0.1772, 0.0231, 0.1973], requires_grad=True)

In [142]:
layer = nn.Linear(in_features=5, out_features=3, bias=False)

In [143]:
layer.bias

In [144]:
layer.__call__

<bound method Module._wrapped_call_impl of Linear(in_features=5, out_features=3, bias=False)>

In [145]:
x = torch.randn(5)

print(layer(x))

tensor([ 0.5503, -0.0826,  0.9584], grad_fn=<SqueezeBackward4>)


In [ ]:
layer1 = nn.Linear(in_features=5, out_features=3)
layer2 = nn.Linear(in_features=3, out_features=1)

layer2(layer1(x))

tensor([-0.2791], grad_fn=<ViewBackward0>)

### Функции активации

> Сигмоида $f(x) = \dfrac{1}{1 + e^{-x}}$

In [147]:
activation = nn.Sigmoid()

In [148]:
x = torch.randn(5)

print(x)

print(activation(x))

tensor([ 0.3339,  0.6522, -0.1555,  0.2271,  1.3149])
tensor([0.5827, 0.6575, 0.4612, 0.5565, 0.7883])


> ReLU $f(x) = \max(0, x)$

In [149]:
activation = nn.ReLU()

In [150]:
x = torch.randn(5)

print(x)

print(activation(x))

tensor([-0.2597,  1.2667,  0.8438,  0.0942,  0.6634])
tensor([0.0000, 1.2667, 0.8438, 0.0942, 0.6634])


> Leaky ReLU $f(x) = \max(0, x) + \alpha \min(0, x)$

In [151]:
activation = nn.LeakyReLU(negative_slope=0.001)

In [152]:
x = torch.randn(5)

print(x)

print(activation(x))

tensor([-0.2619, -0.3646, -1.5159,  0.8460,  0.4465])
tensor([-2.6186e-04, -3.6461e-04, -1.5159e-03,  8.4599e-01,  4.4645e-01])


In [153]:
layer1 = nn.Linear(in_features=5, out_features=3)
activation = nn.LeakyReLU(negative_slope=0.001)
layer2 = nn.Linear(in_features=3, out_features=1)

layer2(activation(layer1(x)))

tensor([0.0441], grad_fn=<ViewBackward0>)

## Градиентный спуск своими руками

In [161]:
n_features = 2
n_objects = 300

torch.manual_seed(0)

w_true = torch.randn(n_features)
b_true = torch.randn(1)

x = (torch.rand(n_objects, n_features) - 0.5) * 10 * (torch.arange(n_features) * 2 + 1)
y = torch.matmul(x, w_true) + torch.randn(n_objects) + b_true

In [155]:
x.shape

torch.Size([300, 2])

In [156]:
y.shape

torch.Size([300])

In [162]:
n_steps = 200
step_size = 1e-2

In [166]:
w = torch.rand(n_features, requires_grad=True)
b = torch.rand(1, requires_grad=True)

print(w)
print(b)
for i in range(n_steps):
    y_pred = torch.matmul(x, w) + b

    mse = torch.mean((y_pred - y) ** 2)

    if i < 20 or i % 10 == 0:
        print(f'MSE на шаге {i + 1} {mse.item():.5f}')

    mse.backward()

#     print(f'{w.grad=}\n')
#     print(f'{b.grad=}\n')

    with torch.no_grad():
        w -= w.grad * step_size
        b -= b.grad * step_size

    w.grad.zero_()
    b.grad.zero_()

tensor([0.9944, 0.0070], requires_grad=True)
tensor([0.0187], requires_grad=True)
MSE на шаге 1 15.86098
MSE на шаге 2 10.01271
MSE на шаге 3 7.67900
MSE на шаге 4 6.55727
MSE на шаге 5 5.90226
MSE на шаге 6 5.45731
MSE на шаге 7 5.12399
MSE на шаге 8 4.85846
MSE на шаге 9 4.63781
MSE на шаге 10 4.44852
MSE на шаге 11 4.28191
MSE на шаге 12 4.13220
MSE на шаге 13 3.99542
MSE на шаге 14 3.86883
MSE на шаге 15 3.75051
MSE на шаге 16 3.63907
MSE на шаге 17 3.53354
MSE на шаге 18 3.43320
MSE на шаге 19 3.33750
MSE на шаге 20 3.24605
MSE на шаге 21 3.15850
MSE на шаге 31 2.45568
MSE на шаге 41 1.98489
MSE на шаге 51 1.66876
MSE на шаге 61 1.45647
MSE на шаге 71 1.31391
MSE на шаге 81 1.21817
MSE на шаге 91 1.15388
MSE на шаге 101 1.11071
MSE на шаге 111 1.08172
MSE на шаге 121 1.06225
MSE на шаге 131 1.04918
MSE на шаге 141 1.04040
MSE на шаге 151 1.03450
MSE на шаге 161 1.03054
MSE на шаге 171 1.02788
MSE на шаге 181 1.02610
MSE на шаге 191 1.02490


In [167]:
layer = nn.Linear(in_features=n_features, out_features=1)

print(layer.weight)
print(layer.bias)

for i in range(n_steps):
    y_pred = layer(x)

    mse = torch.mean((y_pred - y) ** 2)

    if i < 20 or i % 10 == 0:
        print(f'MSE на шаге {i + 1} {mse.item():.5f}')

    mse.backward()

    with torch.no_grad():
        layer.weight -= layer.weight.grad * step_size
        layer.bias -= layer.bias.grad * step_size

#     layer.weight.grad.zero_()
#     layer.bias.grad.zero_()

    layer.zero_grad()

Parameter containing:
tensor([[0.4781, 0.6278]], requires_grad=True)
Parameter containing:
tensor([0.4332], requires_grad=True)
MSE на шаге 1 71.26740
MSE на шаге 2 46.77110
MSE на шаге 3 38.91867
MSE на шаге 4 36.19193
MSE на шаге 5 35.06435
MSE на шаге 6 34.45151
MSE на шаге 7 34.01669
MSE на шаге 8 33.65302
MSE на шаге 9 33.32498
MSE на шаге 10 33.01966
MSE на шаге 11 32.73158
MSE на шаге 12 32.45794
MSE на шаге 13 32.19704
MSE на шаге 14 31.94769
MSE на шаге 15 31.70901
MSE на шаге 16 31.48026
MSE на шаге 17 31.26088
MSE на шаге 18 31.05036
MSE на шаге 19 30.84825
MSE на шаге 20 30.65417
MSE на шаге 21 30.46776
MSE на шаге 31 28.96191
MSE на шаге 41 27.95099
MSE на шаге 51 27.27213
MSE на шаге 61 26.81626
MSE на шаге 71 26.51012
MSE на шаге 81 26.30454
MSE на шаге 91 26.16649
MSE на шаге 101 26.07378
MSE на шаге 111 26.01152
MSE на шаге 121 25.96971
MSE на шаге 131 25.94164
MSE на шаге 141 25.92278
MSE на шаге 151 25.91012
MSE на шаге 161 25.90162
MSE на шаге 171 25.89591
MSE на ша

In [165]:
layer(x)

tensor([[-2.7672],
        [-2.7672],
        [-2.7672],
        [-2.7672],
        [-2.7672],
        [-2.7672],
        [-2.7672],
        [-2.7672],
        [-2.7672],
        [-2.7672],
        [-2.7672],
        [-2.7672],
        [-2.7672],
        [-2.7672],
        [-2.7672],
        [-2.7672],
        [-2.7672],
        [-2.7672],
        [-2.7672],
        [-2.7672],
        [-2.7672],
        [-2.7672],
        [-2.7672],
        [-2.7672],
        [-2.7672],
        [-2.7672],
        [-2.7672],
        [-2.7672],
        [-2.7672],
        [-2.7672],
        [-2.7672],
        [-2.7672],
        [-2.7672],
        [-2.7672],
        [-2.7672],
        [-2.7672],
        [-2.7673],
        [-2.7672],
        [-2.7672],
        [-2.7672],
        [-2.7672],
        [-2.7672],
        [-2.7672],
        [-2.7672],
        [-2.7672],
        [-2.7672],
        [-2.7672],
        [-2.7672],
        [-2.7672],
        [-2.7672],
        [-2.7672],
        [-2.7672],
        [-2.

In [168]:
layer(x).shape

torch.Size([300, 1])

In [174]:
y.shape

torch.Size([300])

In [170]:
(layer(x) - y).shape

torch.Size([300, 300])

In [171]:
layer(x).ravel()

tensor([-2.7104, -2.7095, -2.7134, -2.7124, -2.7074, -2.7120, -2.7094, -2.7152,
        -2.7101, -2.6907, -2.7127, -2.7181, -2.7114, -2.7086, -2.7202, -2.7060,
        -2.7032, -2.6989, -2.7119, -2.7047, -2.7122, -2.7075, -2.7040, -2.6958,
        -2.6999, -2.7115, -2.7053, -2.7163, -2.7005, -2.7087, -2.7075, -2.7135,
        -2.7091, -2.7041, -2.6990, -2.7020, -2.7243, -2.6918, -2.7042, -2.7061,
        -2.6990, -2.7070, -2.7022, -2.7139, -2.7034, -2.7121, -2.7018, -2.7106,
        -2.7180, -2.7104, -2.7092, -2.7164, -2.7208, -2.6938, -2.7106, -2.7076,
        -2.7036, -2.7164, -2.7140, -2.7068, -2.7055, -2.7114, -2.7144, -2.6988,
        -2.7066, -2.7098, -2.7015, -2.6964, -2.7141, -2.6999, -2.7076, -2.7119,
        -2.7137, -2.6945, -2.6949, -2.7070, -2.7083, -2.7113, -2.7094, -2.7153,
        -2.7171, -2.6947, -2.7000, -2.7221, -2.7056, -2.6928, -2.7048, -2.7015,
        -2.7047, -2.7087, -2.7206, -2.7036, -2.7157, -2.7114, -2.7165, -2.7140,
        -2.7117, -2.7110, -2.7190, -2.71

In [172]:
(layer(x).ravel() - y).shape

torch.Size([300])

In [175]:
layer = nn.Linear(in_features=n_features, out_features=1)

for i in range(n_steps):
    y_pred = layer(x).ravel()

    mse = torch.mean((y_pred - y) ** 2)

    if i < 20 or i % 10 == 0:
        print(f'MSE на шаге {i + 1} {mse.item():.5f}')

    mse.backward()

    with torch.no_grad():
        layer.weight -= layer.weight.grad * step_size
        layer.bias -= layer.bias.grad * step_size

    layer.zero_grad()

MSE на шаге 1 60.51762
MSE на шаге 2 38.10868
MSE на шаге 3 25.79010
MSE на шаге 4 18.25825
MSE на шаге 5 13.35753
MSE на шаге 6 10.06214
MSE на шаге 7 7.80691
MSE на шаге 8 6.24670
MSE на шаге 9 5.15782
MSE на шаге 10 4.39096
MSE на шаге 11 3.84500
MSE на шаге 12 3.45099
MSE на шаге 13 3.16180
MSE на шаге 14 2.94515
MSE на шаге 15 2.77890
MSE на шаге 16 2.64787
MSE на шаге 17 2.54164
MSE на шаге 18 2.45303
MSE на шаге 19 2.37711
MSE на шаге 20 2.31048
MSE на шаге 21 2.25079
MSE на шаге 31 1.83216
MSE на шаге 41 1.56585
MSE на шаге 51 1.38735
MSE на шаге 61 1.26749
MSE на шаге 71 1.18700
MSE на шаге 81 1.13295
MSE на шаге 91 1.09666
MSE на шаге 101 1.07228
MSE на шаге 111 1.05591
MSE на шаге 121 1.04492
MSE на шаге 131 1.03754
MSE на шаге 141 1.03258
MSE на шаге 151 1.02925
MSE на шаге 161 1.02702
MSE на шаге 171 1.02552
MSE на шаге 181 1.02451
MSE на шаге 191 1.02383


In [176]:
n_features = 5
n_objects = 300

torch.manual_seed(0)

w_true = torch.randn(n_features)

x = (torch.rand(n_objects, n_features) - 0.5) * 10 * (torch.arange(n_features) * 2 + 1)
y = torch.matmul(x, w_true) + torch.randn(n_objects)

In [177]:
x.shape, y.shape

(torch.Size([300, 5]), torch.Size([300]))

In [178]:
n_steps = 500
step_size = 1e-3

In [179]:
layer = nn.Linear(in_features=n_features, out_features=1)

for i in range(n_steps):
    y_pred = layer(x).ravel()

    mse = torch.mean((y_pred - y) ** 2)

    if i < 20 or i % 50 == 0:
        print(f'MSE на шаге {i + 1} {mse.item():.5f}')

    mse.backward()

    with torch.no_grad():
        layer.weight -= layer.weight.grad * step_size
        layer.bias -= layer.bias.grad * step_size

    layer.zero_grad()

MSE на шаге 1 1522.64282
MSE на шаге 2 345.22870
MSE на шаге 3 134.71320
MSE на шаге 4 69.14159
MSE на шаге 5 45.61798
MSE на шаге 6 36.15922
MSE на шаге 7 31.74179
MSE на шаге 8 29.24492
MSE на шаге 9 27.53952
MSE на шаге 10 26.19924
MSE на шаге 11 25.05364
MSE на шаге 12 24.02866
MSE на шаге 13 23.08827
MSE на шаге 14 22.21260
MSE на шаге 15 21.38941
MSE на шаге 16 20.61047
MSE на шаге 17 19.86992
MSE на шаге 18 19.16344
MSE на шаге 19 18.48770
MSE на шаге 20 17.84015
MSE на шаге 51 6.27402
MSE на шаге 101 1.74083
MSE на шаге 151 1.02593
MSE на шаге 201 0.91269
MSE на шаге 251 0.89435
MSE на шаге 301 0.89104
MSE на шаге 351 0.89019
MSE на шаге 401 0.88977
MSE на шаге 451 0.88948


In [180]:
n_steps = 1000
step_size = 3e-4

In [181]:
layer1 = nn.Linear(in_features=n_features, out_features=3)
layer2 = nn.Linear(in_features=3, out_features=1)
activation = nn.ReLU()


for i in range(n_steps):
    y_pred = layer2(activation(layer1(x))).ravel()

    mse = torch.mean((y_pred - y) ** 2)

    if i < 20 or i % 50 == 0:
        print(f'MSE на шаге {i + 1} {mse.item():.5f}')

    mse.backward()

    with torch.no_grad():
        layer1.weight -= layer1.weight.grad * step_size
        layer1.bias -= layer1.bias.grad * step_size
        layer2.weight -= layer2.weight.grad * step_size
        layer2.bias -= layer2.bias.grad * step_size

    layer1.zero_grad()
    layer2.zero_grad()

MSE на шаге 1 2065.53369
MSE на шаге 2 1941.40039
MSE на шаге 3 1890.73816
MSE на шаге 4 1863.17334
MSE на шаге 5 1843.74536
MSE на шаге 6 1825.21838
MSE на шаге 7 1803.15234
MSE на шаге 8 1775.96960
MSE на шаге 9 1741.44421
MSE на шаге 10 1696.50024
MSE на шаге 11 1638.66479
MSE на шаге 12 1570.60828
MSE на шаге 13 1498.17651
MSE на шаге 14 1426.69543
MSE на шаге 15 1361.06189
MSE на шаге 16 1304.04346
MSE на шаге 17 1252.44177
MSE на шаге 18 1201.12524
MSE на шаге 19 1142.44934
MSE на шаге 20 1069.69470
MSE на шаге 51 10.77816
MSE на шаге 101 5.31390
MSE на шаге 151 3.22006
MSE на шаге 201 2.08225
MSE на шаге 251 1.45981
MSE на шаге 301 1.16964
MSE на шаге 351 1.03444
MSE на шаге 401 0.97123
MSE на шаге 451 0.94165
MSE на шаге 501 0.92668
MSE на шаге 551 0.91742
MSE на шаге 601 0.91137
MSE на шаге 651 0.90724
MSE на шаге 701 0.90359
MSE на шаге 751 0.90093
MSE на шаге 801 0.89879
MSE на шаге 851 0.89742
MSE на шаге 901 0.89643
MSE на шаге 951 0.89556
